In [19]:
import math

def haversine(src, des):
    lat1, lon1 = src[0], src[1]
    lat2, lon2 = des[0], des[1]
    R = 6371000
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # Distance in meters

In [27]:
# clockwise
def calculate_bearing(src, des):
    lat1, lon1 = src[0], src[1]
    lat2, lon2 = des[0], des[1]
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    delta_lon = lon2 - lon1

    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon))
    initial_bearing = math.atan2(x, y)

    initial_bearing = math.degrees(initial_bearing)

    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing


In [25]:
print(calculate_bearing([25.4358, 81.8463], [24.5362, 81.3037]))

208.78081401749466


In [3]:
print(haversine(25.4358, 81.8463, 24.5362, 81.3037))

114003.6627918423


In [9]:
def time_taken(distance, speed=10):
    return distance / speed  # Time in seconds

In [13]:
import json

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)  # Load the file into a Python dictionary
        return data
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON from {file_path}.")
        return None

In [36]:
def populate_coord_list(file_path):
    json_file = load_json_file(file_path)
    ret = json_file['routes'][0]['geometry']['coordinates']
    return ret

In [60]:
from datetime import datetime, timedelta
date_string = "1/10/24 09:00"
date_format = "%d/%m/%y %H:%M"  # Format for parsing the date
initial_time = datetime(2024, 10, 1, 9, 0)

In [38]:
soha_list = populate_coord_list("./soha.json")

In [44]:
print(len(soha_list))

603


In [74]:
def generate_route_data(file_path, speed=10):
    coordinates = populate_coord_list(file_path)
    route_data = []

    if not coordinates:
        return route_data  # Return an empty list if no coordinates

    # Set the initial timestamp for the first coordinate
    current_time = initial_time

    # Loop through coordinates, calculating bearing and timestamps
    for i in range(len(coordinates) - 1):
        coordinate = coordinates[i]
        bearing = calculate_bearing(coordinates[i], coordinates[i + 1])
        
        # Calculate the distance between the current and next coordinate
        distance = haversine(coordinates[i], coordinates[i + 1])
        # Calculate time taken based on the distance and speed
        time_interval = time_taken(distance, speed)
        # Update the current time
        current_time += timedelta(seconds=time_interval)
        
        # Append dictionary to the result list with the current coordinate
        route_data.append({
            'coordinate': coordinate,
            'bearing': bearing,
            'timestamp': current_time.isoformat()
        })

    # Append the last coordinate with its bearing and timestamp
    last_coordinate = coordinates[-1]
    last_bearing = calculate_bearing(coordinates[-2], last_coordinate) if len(coordinates) > 1 else None
    route_data.append({
        'coordinate': last_coordinate,
        'bearing': last_bearing,
        'timestamp': current_time.isoformat()
    })

    return route_data

In [90]:
soha_final = generate_route_data("./soha.json", 20)

In [95]:
retDictSoha = {
    'points': soha_final, 
    'metadata': {
        'title' : 'PRYG - REWA RD',
        'speed' : 20
    }
}

In [97]:
with open('soha_final.json', 'w') as json_file:
    json.dump(retDictSoha, json_file, indent=4)

In [86]:
soha_final = generate_route_data("./soha.json", 10)

603


In [101]:
civil_final = generate_route_data("./civil.json", 10)

retDictCivil = {
    'points': civil_final, 
    'metadata': {
        'title' : 'Civil Lines sq',
        'speed' : 10
    }
}

with open('civil_final.json', 'w') as json_file:
    json.dump(retDictCivil, json_file, indent=4)